In [1]:
import requests
import os
import pandas as pd

In [2]:

#Obtenemos la lista de subcarpetas desde archivo RECORDS
records_url = "https://physionet.org/files/ecg-arrhythmia/1.0.0/RECORDS"
resp = requests.get(records_url)
resp.raise_for_status()
lines = resp.text.splitlines()

#Filtrar las rutas tipo "WFDBRecords/01/010/"
paths = [line.strip() for line in lines if line.strip().startswith("WFDBRecords/")]

file_list = []

#Para cada ruta, obtener archivos listados en RECORDS local de ese subdirectorio
for p in paths:
    url = f"https://physionet.org/files/ecg-arrhythmia/1.0.0/{p}/RECORDS"
    sub_resp = requests.get(url)
    if sub_resp.status_code != 200:
        continue  # saltar si no hay RECORDS en ese subdirectorio
    entries = sub_resp.text.splitlines()
    for rec in entries:
        hea_name = f"{rec}"
        full_url = f"https://physionet.org/files/ecg-arrhythmia/1.0.0/{p}{hea_name}"
        # Añadimos nombre y ruta
        file_list.append((int(hea_name[2:7]),hea_name, full_url))

In [3]:
#Pasar a DataFrame
dfListado = pd.DataFrame(file_list, columns=["Id","Paciente", "Archivo"])

#Guardar a CSV
dfListado.to_csv("Directorio-ecg.csv", index=False, encoding="utf-8")
